# Сбор и анализ отзывов о ресторанах на tripadvisor.ru

## Crawler

В проекте, вооружившись статьями из разных уважаемых источников (например, ["Ведомости"](https://www.vedomosti.ru/lifestyle/articles/2016/03/25/637995-vesna-v-restoranah-moskvi),) я решила посмотреть, можно ли, обработав данные о московских ресторанах с известными средним чеком и датой первого отзыва на сайте tripadvisor.com, сделать вывод о том, что в кризис посетители стали предпочитать более бюджетные рестораны

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import time
import datetime
import urllib
import datetime
import numpy as np
import matplotlib.pyplot as plt
import csv
import xlsxwriter
writer = pd.ExcelWriter('reviews.xlsx', engine='xlsxwriter')
%matplotlib inline

In [ ]:
# Загрузить стартовую страницу и подготовить файл для записи
driver = webdriver.Chrome('C:/Users/Daria/Documents/chromedriver.exe')
ref = 'https://www.tripadvisor.com/Restaurants-g298484-Moscow_Central_Russia.html#MAINWRAP'
driver.get(ref)
filters = ['//*[@id="jfy_filter_bar_price"]/div[2]/div[1]',
           '//*[@id="jfy_filter_bar_price"]/div[2]/div[2]',
           '//*[@id="jfy_filter_bar_price"]/div[2]/div[3]']
for f in filters:
    i = 0
    while not 'selected' in driver.find_element_by_xpath(f).get_attribute('class') or i > 10:
        driver.find_element_by_xpath(f).click()
        time.sleep(1)
        i += 1
source = driver.page_source

In [ ]:
# Функция выгружает рестораны с ненулевым количеством отзывов, получает имя ресторана, средний чек и ссылку на страницу с отзывами
def download_rests_data(page, file):
    soup = bs(page, 'lxml')
    divs = soup.findAll('div', class_='shortSellDetails')
    # Удалить рестораны с пустыми отзывами - exception из-за разной двух вариантов структуры тэгов с нулевыми отзывами
    rests_with_reviews = []
    for div in divs:
        try:
            if div.find('div', class_ = "rating").a.text.strip() != 'Оставить первый отзыв об этом ресторане':
                rests_with_reviews.append(div)
        except:
            pass
    # Получить имя ресторана
    names = [div.find('h3', class_='title').text.strip() for div in rests_with_reviews]
    print(len(names))
    # Получить средний чек
    checks = []
    for div in rests_with_reviews:
        try:
            check = div.find('span', class_='price').text.strip()
        except:
            check = 'NaN'
        checks.append(check)
    # Получить ссылки на отзывы
    hrefs_raw = [div.find('span', class_='reviewCount') for div in rests_with_reviews]
    hrefs = ['https://www.tripadvisor.ru' + i.a['href'] for i in hrefs_raw]
    # дописать в файл
    database = pd.DataFrame(data = list(zip(names, checks, hrefs)))
    with open(file, 'a') as f:
        database.to_csv(f, encoding='unicode', header = False, index=False)

In [ ]:
# Проходим по выданным фильтром страницам и собираем начальную информацию
page_num = 1
position = 30
last_page = int(driver.find_element_by_css_selector("#EATERY_LIST_CONTENTS > div.deckTools.btm > div > div > a:nth-child(8)").text)
print(last_page)
while page_num <= 5:
    href = 'https://www.tripadvisor.ru/RestaurantSearch-g298484-oa' + str(position) + '-p15-Moscow_Central_Russia.html#EATERY_LIST_CONTENTS'
    driver.get(href)
    source = driver.page_source
    download_rests_data(source, 'tripadvisor_restaurants.csv')
    print(page_num, "DONE")
    page_num = page_num + 1
    position = position + 30

In [ ]:
# Открываем получившийся CSV в виде датафрейма, добавляем хэдеры
database = pd.read_csv('tripadvisor_restaurants.csv', header = None, names = ['Name', 'Avg check', 'Hrefs'], encoding = 'cp1251')

In [ ]:
database[1:11]

Тут средний чек отобразился неправильно из-за встроенной кодировки, но в самом csv-файле все в порядке

In [ ]:
def get_revs_data(page):
    driver.get(page)
    try:
        last_page = int(driver.find_element_by_xpath(("//a[@class='pageNum taLnk' and contains(@onclick,'last')]")).text)
    except:
        last_page = 1
    months = {' января ':'01', ' февраля ':'02', ' марта ':'03', ' апреля ':'04', ' мая ':'05', ' июня ':'06', ' июля ':'07', ' августа ':'08',
            ' сентября ':'09', ' октября ':'10', ' ноября ':'11', ' декабря ':'12'}
    pattern = re.compile(r'\b(' +'|'.join(months.keys())+ r')\b')
    page_num = 1
    position = 10
    dates = []
    ratings = []
    while page_num <= last_page:
        if page_num == 1:
            rev_href = page
        else:
            rev_href = re.sub(r'^((.*?-.*?){2})-', r'\1-or%s-' %position, page)
            if page_num < last_page:
                position += 10
            driver.get(rev_href)
        rest_soup = bs(driver.page_source,'lxml')
        review_bubbles = rest_soup.findAll('div', class_='reviewItemInline')
        for bubble in review_bubbles:
            # Получим даты отзывов
            raw_date1 = bubble.find('span', class_='ratingDate')
            # Из-за неоднородной структуры страниц несколько исключений
            try:
                raw_date1.string.replace_with(raw_date1.string.strip())
            except:
                raw_date1.previousSibling.replace_with(raw_date1.previousSibling.strip())
            try:
                date = raw_date1['title']
            except:
                date = raw_date1.text.split(' ', 2)[-1]
            raw_date2 = pattern.sub(lambda x: months[x.group()], date)
                # В некоторых случаях strptime не парсит даты, например '3122013' возвращает ошибку out of range
            if len(raw_date2) == 7:
                raw_date2 = '0' + raw_date2
            date_fine = datetime.datetime.strptime(raw_date2, "%d%m%Y").date()
            dates.append(date_fine)
                # Собираем рейтинги
            rating = int(bubble.img['alt'].split(' ',1)[0])
            ratings.append(rating)
        print(page_num, ' done')
        page_num += 1
    stat = pd.DataFrame({'Оценка':ratings, 'Дата':dates})
    return stat

In [ ]:
stats = []
name = database['Name'][database['Hrefs'] == 'https://www.tripadvisor.ru/Restaurant_Review-g298484-d6495969-Reviews-Kitayskaya_Gramota-Moscow_Central_Russia.html#REVIEWS']
name.values[0]

In [ ]:
# Для каждого ресторана заходим в href, проходимся по ссылкам и собираем дату и рейтинг для каждого отзыва
# Записываем даты и рейтинги в structured numpy array
# Добавляем в датафрейм column и помещаем туда массив для каждого ресторана
for href in database['Hrefs']:
    name = database['Name'][database['Hrefs'] == href]
    print(name)
    revs = get_revs_data(href)
    revs.to_excel(writer, sheet_name = str(name.values[0]))
    print(name, 'DONE')
writer.save()

In [ ]:
writer.save()

## Старый код и графики